### Pyspark Initiation and Setting

In [1]:
# Import SparkConf class into program
from pyspark import SparkConf

# local[*]: run Spark in local mode with as many working processors as logical cores on your machine
# If we want Spark to run locally with 'k' worker threads, we can specify as "local[k]".
master = "local[*]"
# The `appName` field is a name to be shown on the Spark cluster UI page
app_name = "Assignment 2"
# Setup configuration parameters for Spark
spark_conf = SparkConf().setMaster(master).setAppName(app_name)

### SparkContext and SparkSession

In [2]:
# Import SparkContext and SparkSession classes
from pyspark import SparkContext # Spark
from pyspark.sql import SparkSession # Spark SQL

# Using SparkSession
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('ERROR')

In [3]:
# Importing the required libraries

from pyspark import SparkConf # Spark
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml import Pipeline

### Step 1: Data Loading and Preparation

In [4]:
# Optimisation step - include schema definitions first before reading in data
from pyspark.sql.types import *

schema = StructType([
    StructField('ID', IntegerType(), True),
    StructField('Type', StringType(), True),
    StructField('Rooms', IntegerType(), True), 
    StructField('Year', IntegerType(), True), 
    StructField('Suburb', StringType(), True),
    StructField('Address', StringType(), True),
    StructField('Price', DoubleType(), True), 
    StructField('Method', StringType(), True), 
    StructField('SellerG', StringType(), True), 
    StructField('Distance', IntegerType(), True),
    StructField('Landsize', DoubleType(), True), 
    StructField('CouncilArea', StringType(), True), 
    StructField('Regionname', StringType(), True),
    StructField('Month', IntegerType(), True), 
    StructField('Day', IntegerType(), True), 
    StructField('MedianPrice', DoubleType(), True), 
    StructField('AuctionResult', IntegerType(), True),
    StructField('StreetSuffix', StringType(), True), 
    StructField('binnedDistance', IntegerType(), True)
])

file = "Dataset_for_Model_1_Price_Prediction.csv"

df = spark.read.csv(file,  
    header=True,
    schema=schema
)
df.show(1)

+---+----+-----+----+----------+-------------+--------+------+-------+--------+--------+--------------------+--------------------+-----+---+-----------+-------------+------------+--------------+
| ID|Type|Rooms|Year|    Suburb|      Address|   Price|Method|SellerG|Distance|Landsize|         CouncilArea|          Regionname|Month|Day|MedianPrice|AuctionResult|StreetSuffix|binnedDistance|
+---+----+-----+----+----------+-------------+--------+------+-------+--------+--------+--------------------+--------------------+-----+---+-----------+-------------+------------+--------------+
|  0|   t|    3|2018|point cook|23 Tribeca Dr|535000.0|    SA|  Point|      14|    78.0|wyndham city council|western metropolitan|    2| 17|   592000.0|            0|          Dr|             2|
+---+----+-----+----+----------+-------------+--------+------+-------+--------+--------+--------------------+--------------------+-----+---+-----------+-------------+------------+--------------+
only showing top 1 row



In [5]:
# Print the schema in tree format 
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Rooms: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Suburb: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Method: string (nullable = true)
 |-- SellerG: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- Landsize: double (nullable = true)
 |-- CouncilArea: string (nullable = true)
 |-- Regionname: string (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- MedianPrice: double (nullable = true)
 |-- AuctionResult: integer (nullable = true)
 |-- StreetSuffix: string (nullable = true)
 |-- binnedDistance: integer (nullable = true)



###  Remove nulls

This dataset has nulls remaining in it because it will be used for two different models, and we believe that different features will be of varying usefulness for each one (basically we did't want to delete data on account of a column that may not even be used in the end).

In [6]:
# Checking for the null values

from pyspark.sql.functions import isnan, when, count, col

nulls = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).collect()

In [7]:
# Look at the missing values across the columns

print("Missing values:\n")
for i in range(len(df.columns)):
    print(f"{df.columns[i]}: {nulls[0][i]}")

Missing values:

ID: 0
Type: 0
Rooms: 0
Year: 0
Suburb: 0
Address: 0
Price: 854
Method: 0
SellerG: 0
Distance: 0
Landsize: 0
CouncilArea: 0
Regionname: 0
Month: 0
Day: 0
MedianPrice: 267
AuctionResult: 0
StreetSuffix: 0
binnedDistance: 0


In [8]:
# Drop the nulls

df = df.na.drop()


In [9]:
# Checking the dataframe after dropping null values

df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+----+-----+----+------+-------+-----+------+-------+--------+--------+-----------+----------+-----+---+-----------+-------------+------------+--------------+
| ID|Type|Rooms|Year|Suburb|Address|Price|Method|SellerG|Distance|Landsize|CouncilArea|Regionname|Month|Day|MedianPrice|AuctionResult|StreetSuffix|binnedDistance|
+---+----+-----+----+------+-------+-----+------+-------+--------+--------+-----------+----------+-----+---+-----------+-------------+------------+--------------+
|  0|   0|    0|   0|     0|      0|    0|     0|      0|       0|       0|          0|         0|    0|  0|          0|            0|           0|             0|
+---+----+-----+----+------+-------+-----+------+-------+--------+--------+-----------+----------+-----+---+-----------+-------------+------------+--------------+



### Dtypes

In [10]:
df.dtypes

[('ID', 'int'),
 ('Type', 'string'),
 ('Rooms', 'int'),
 ('Year', 'int'),
 ('Suburb', 'string'),
 ('Address', 'string'),
 ('Price', 'double'),
 ('Method', 'string'),
 ('SellerG', 'string'),
 ('Distance', 'int'),
 ('Landsize', 'double'),
 ('CouncilArea', 'string'),
 ('Regionname', 'string'),
 ('Month', 'int'),
 ('Day', 'int'),
 ('MedianPrice', 'double'),
 ('AuctionResult', 'int'),
 ('StreetSuffix', 'string'),
 ('binnedDistance', 'int')]

#### Features must be defined as numeric or categoric so they can be treated appropriately by the data pipeline.

In [11]:
categoryInputCols = ['Type', "Suburb", "Method", "SellerG", "CouncilArea", "Regionname", "StreetSuffix"]
numericInputCols = ["Rooms", "Year", "Distance", "Landsize", "Month", "Day", "MedianPrice", "AuctionResult"]

# The response var
numericOutputCol = 'Price'

categoryCols = categoryInputCols
numericCols = numericInputCols+[numericOutputCol]

### Step 2: Feature Engineering 

#### OHE: To convert categorical values into numeric values, we use a technique called one-hot encoding (OHE).

#### Indexer: A label indexer that maps a string column of labels to an ML column of label indices. 

If the input column is numeric, we cast it to integer and index the string values. The indices are in [0, numLabels). By default, this is ordered by label frequencies so the most frequent label gets index 0. 

In [12]:
# Defining the output columns

outputCols=[f'{x}_index' for x in categoryInputCols]
inputIndexer = StringIndexer(inputCols=categoryCols, outputCols=outputCols).setHandleInvalid("keep")

Test results:

In [13]:
# apply indexing to categoric data
df_indexed = inputIndexer.fit(df).transform(df)

# show encoded result
df_indexed.select(outputCols).show(2)

+----------+------------+------------+-------------+-----------------+----------------+------------------+
|Type_index|Suburb_index|Method_index|SellerG_index|CouncilArea_index|Regionname_index|StreetSuffix_index|
+----------+------------+------------+-------------+-----------------+----------------+------------------+
|       2.0|        84.0|         6.0|         96.0|             19.0|             2.0|               4.0|
|       2.0|        84.0|         0.0|         96.0|             19.0|             2.0|              13.0|
+----------+------------+------------+-------------+-----------------+----------------+------------------+
only showing top 2 rows



#### Encoder

 Encoders are highly specialized and optimized code generators that generate custom bytecode for serialization and deserialization of the data.

In [14]:

inputCols_OHE = [x for x in outputCols if x!='label']
outputCols_OHE = [f'{x}_vec' for x in categoryInputCols]

# Define OneHotEncoder with the appropriate columns

encoder = OneHotEncoder(inputCols=inputCols_OHE,outputCols=outputCols_OHE)

Test result:

In [15]:
model = encoder.fit(df_indexed)
# Call fit and transform to get the encoded results
df_encoded = model.transform(df_indexed)
# Display the output columns
df_encoded.select(outputCols_OHE).show(2)

+-------------+----------------+-------------+----------------+---------------+--------------+----------------+
|     Type_vec|      Suburb_vec|   Method_vec|     SellerG_vec|CouncilArea_vec|Regionname_vec|StreetSuffix_vec|
+-------------+----------------+-------------+----------------+---------------+--------------+----------------+
|(3,[2],[1.0])|(324,[84],[1.0])|(9,[6],[1.0])|(363,[96],[1.0])|(33,[19],[1.0])| (8,[2],[1.0])|  (22,[4],[1.0])|
|(3,[2],[1.0])|(324,[84],[1.0])|(9,[0],[1.0])|(363,[96],[1.0])|(33,[19],[1.0])| (8,[2],[1.0])| (22,[13],[1.0])|
+-------------+----------------+-------------+----------------+---------------+--------------+----------------+
only showing top 2 rows



#### Assembler: A feature transformer that merges multiple columns into a vector column.

In [16]:
# inputCols are all the encoded columns from OHE plus numerical columns
inputCols=outputCols_OHE
assemblerInputs = outputCols_OHE + numericInputCols

# Define the assembler with appropriate input and output columns
assembler = VectorAssembler(inputCols = assemblerInputs, outputCol="features")

Check result:

In [17]:
# use the asseembler transform() to get encoded results
df_assembled = assembler.transform(df_encoded)

# Display the output
df_assembled.select('features').show()

+--------------------+
|            features|
+--------------------+
|(770,[2,87,333,43...|
|(770,[2,87,327,43...|
|(770,[2,87,329,39...|
|(770,[0,40,327,36...|
|(770,[0,40,327,33...|
|(770,[0,40,327,33...|
|(770,[0,40,328,33...|
|(770,[0,40,327,34...|
|(770,[0,40,327,33...|
|(770,[0,40,329,33...|
|(770,[0,40,327,33...|
|(770,[0,40,327,33...|
|(770,[0,40,327,33...|
|(770,[0,40,327,33...|
|(770,[0,40,329,33...|
|(770,[0,40,327,33...|
|(770,[0,40,327,33...|
|(770,[0,40,330,33...|
|(770,[0,40,329,33...|
|(770,[0,40,330,33...|
+--------------------+
only showing top 20 rows



### StandardScaler

Standardizes features by removing the mean and scaling to unit variance using column summary statistics on the samples in the training set.

In [18]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
standard_scaler = StandardScaler(inputCol="features", outputCol="scaled")

Test results:

In [19]:
df_scaled = standard_scaler.fit(df_assembled).transform(df_assembled)

In [20]:
df_scaled.select('scaled').show(5)

+--------------------+
|              scaled|
+--------------------+
|(770,[2,87,333,43...|
|(770,[2,87,327,43...|
|(770,[2,87,329,39...|
|(770,[0,40,327,36...|
|(770,[0,40,327,33...|
+--------------------+
only showing top 5 rows



### Step 3: Pipeline

A Pipeline chains multiple Transformers and Estimators together to specify an ML workflow.

In [21]:
# Pipelines and PipelineModels help to ensure that training and test data go through identical feature processing steps.
stage_1 = inputIndexer
stage_2 = encoder
stage_3 = assembler
stage_4 = standard_scaler


In [22]:
#stages = [stage_1,stage_2,stage_3, stage_4, stage_5]
stages = [inputIndexer,encoder,assembler,standard_scaler ]

pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df)
df_pipeline = pipelineModel.transform(df)

### Step 4: Creating Training and Test Data Sets

In [23]:
# Divide data into train sets and test sets. 
# Seed is the value used to make the same data three times later
train_df, test_df = df_pipeline.randomSplit([.7, .3], seed=42)
print(f"""There are {train_df.count()} rows in the training set,
and {test_df.count()} in the test set""")

There are 22097 rows in the training set,
and 9397 in the test set


### Linear Regression

Linear regression models a linear relationship between the dependent variable (or
label) and one or more independent variables (or features).

In [40]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol='scaled', labelCol='Price', ) 
lrModel = lr.fit(train_df)
#lrModel
#print("Coefficients: " + str(lrModel.coefficients))
#print("Intercept: " + str(lrModel.intercept))

In [39]:
# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 345780.558716
r2: 0.730503


#### Linear Regression Line

- To fit a linear regression model to predict the price of the house given the number of rooms.
- we have a single feature x and an output y (this is our dependent variable).
- Linear regression seeks to fit an equation for a line to x and y, which for scalar variables can be expressed as y = mx + b, where m is the slope and b is the offset or intercept.

In [29]:
# Creating the linear regression line
m = round(lrModel.coefficients[0], 2)
b = round(lrModel.intercept, 2)
print(f"""The formula for the linear regression line is
price = {m}*Rooms + {b}""")

The formula for the linear regression line is
price = 92777.33*Rooms + -142924521.32


The negative intercept value indicates that when we extend the regression line downwards until we reach the point where it crosses the y-axis and also it is difficult to interpret the constant term because the y-intercept is almost meaningless, yet it is always vital to include it in regression models

#### Testing the Model

In [41]:
# Testing the model on test data set
lrPredictions = lrModel.transform(test_df)

In [42]:
lrPredictions.select('scaled','Price','prediction').show(10)

+--------------------+---------+-----------------+
|              scaled|    Price|       prediction|
+--------------------+---------+-----------------+
|(770,[2,87,329,39...| 597817.0|97148.55679512024|
|(770,[0,40,328,33...| 900000.0|709556.1500967145|
|(770,[0,40,327,33...|1315000.0| 879538.629814595|
|(770,[0,40,329,33...| 440000.0|785428.0889554322|
|(770,[0,40,327,33...|1051000.0|924063.7782126069|
|(770,[0,40,329,33...| 720000.0|912311.8418701887|
|(770,[0,40,327,33...| 590069.0|935910.1229119599|
|(770,[0,40,330,33...| 545000.0|829006.8728214502|
|(770,[0,40,330,33...| 465000.0|904364.2704406679|
|(770,[0,40,329,33...| 825000.0|800162.2443299592|
+--------------------+---------+-----------------+
only showing top 10 rows



### Model Evaluation

Now that we have built a model, we need to evaluate how well it performs. In spark.ml there are classification, regression, clustering, and ranking evaluators. Given that this is a regression problem, we will use rootmean- square error (RMSE) and R2 (pronounced “R-squared”) to evaluate our model’s performance.

#### R2 (R-Squared)

In [59]:
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="Price",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lrPredictions))

R Squared (R2) on test data = 0.691328


R squared at 0.69 indicates that in our model, approximate 69% of the variability in “Price” can be explained using the model.

#### RMSE
Evaluator for Regression, which expects input columns prediction, label and an optional weight column. RMSE is a metric that ranges from zero to infinity. 

In [60]:
test_result = lrModel.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 369786


## Cross Validation and Hyperparameter Tuning

We will fine tune the models using grid search technique which will evaluate all the possible combinations of hyperparameters values using cross validation.

In [61]:
# Defining the pipeline estimator
pipeline = Pipeline(stages = [inputIndexer,encoder,assembler, standard_scaler, lr])

In [62]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator,CrossValidatorModel
from pyspark.ml.evaluation import RegressionEvaluator
# Create ParamGrid for Cross Validation
# Specifying the hyperparameters and their respective values using the ParamGridBuilder
paramGrid = (ParamGridBuilder()
.addGrid(lr.maxIter, [1000])
.addGrid(lr.regParam, [0.1,0.5, 1.0, 2.0])
.addGrid(lr.elasticNetParam, [0.3,0.6, 0.9])
.build())


In [63]:
# Defining how to evaluate each of the models to determine which one perfroms best
# Using RegressionEvaluator and RMSE as the metric
evaluator = RegressionEvaluator(labelCol="Price",
predictionCol="prediction",
metricName="rmse")
 

We will perform the k-fold cross-validation using the CrossValidator, which accepts an estimator, evaluator, and estimatorParamMaps so that it knows which model to use, how to evaluate the model, and which hyperparameters to set for the model. We also set the number of folds we want to split the data into (numFolds=3).

In [64]:
# Cross validator
from pyspark.ml.tuning import CrossValidator
cv = CrossValidator(estimator=lr,
evaluator=evaluator,
estimatorParamMaps=paramGrid,
numFolds=3,
parallelism = 4)

In [65]:
# Fitting the cvModel to train data set
cvModel = cv.fit(train_df)
print(cvModel)

CrossValidatorModel_6a52593c4b67


In [66]:
bestModel = cvModel.bestModel

In [67]:
# Testing the model on test data set
lrPredictions = bestModel.transform(test_df)
lrPredictions.select('scaled','Price','prediction').show(10)

+--------------------+---------+-----------------+
|              scaled|    Price|       prediction|
+--------------------+---------+-----------------+
|(770,[2,87,329,39...| 597817.0| 98250.3813854903|
|(770,[0,40,328,33...| 900000.0|710048.8479898721|
|(770,[0,40,327,33...|1315000.0|879680.9555134624|
|(770,[0,40,329,33...| 440000.0|786156.5398058295|
|(770,[0,40,327,33...|1051000.0|924506.0409045517|
|(770,[0,40,329,33...| 720000.0| 912652.246259138|
|(770,[0,40,327,33...| 590069.0|936321.5586847812|
|(770,[0,40,330,33...| 545000.0|829428.3790077269|
|(770,[0,40,330,33...| 465000.0| 904236.783978954|
|(770,[0,40,329,33...| 825000.0|800112.5389250219|
+--------------------+---------+-----------------+
only showing top 10 rows



In [68]:
from pyspark.ml.evaluation import RegressionEvaluator
regressionEvaluator = RegressionEvaluator(
predictionCol="prediction",
labelCol="Price",
metricName="rmse")
rmse = regressionEvaluator.evaluate(lrPredictions)
print(f"RMSE is {rmse:.1f}")

RMSE is 369802.8


In [55]:
# Calculating the R-squared value
r2 = regressionEvaluator.setMetricName("r2").evaluate(lrPredictions)
print(f"R2 is {r2}")

R2 is 0.6913281685948564


#### Cross Validation and Hyperparameter Tuning methods didn't improve the Linear Regression Model performance